In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import ensembleReader as er
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import keras_models.dataGen as dg
import keras_models.cnn_models as cnn_models
reload(dg)
import predictGenderConvExp as pg
from addict import Dict
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Activation, Conv2D, Concatenate, Dropout, MaxPool2D, Conv3D, Flatten, LeakyReLU, BatchNormalization

In [3]:
files, genders = cta.demux_to_tokens(cta.getGenderAndFileNames("combined", "01_tcp_ar", True))

In [4]:
sessions = [read.parse_edf_token_path_structure(file)[2] for file in files]
patients = [read.parse_edf_token_path_structure(file)[1] for file in files]

In [5]:
allPatients = list(set(patients))

In [6]:
allPatients.sort()

In [7]:
len(allPatients)

318

In [8]:
sessionDict = Dict()
for i, file in enumerate(files):
    sessionDict[patients[i]][sessions[i]][i].file = file  
    sessionDict[patients[i]][sessions[i]][i].gender = genders[i]
    sessionDict[patients[i]][sessions[i]][i].patient = allPatients.index(patients[i])


In [9]:
#delete all patients with only one session
for patient in set(patients):
    print(len(sessionDict[patient].keys()))    
    if len(sessionDict[patient].keys()) < 2:
        del sessionDict[patient]


6
1
2
1
1
1
2
4
5
3
1
1
1
3
1
1
1
1
1
2
1
1
5
3
1
2
2
1
4
1
1
1
1
1
1
3
4
3
1
9
7
1
1
1
1
6
3
1
3
1
1
1
1
4
1
2
6
2
1
6
1
8
4
1
2
7
1
2
1
1
2
1
5
2
2
3
8
2
1
2
2
1
1
3
1
2
3
3
1
1
2
2
1
1
2
1
1
4
1
2
1
1
1
1
3
1
1
1
1
9
1
1
1
1
9
1
1
1
1
1
5
4
1
1
1
1
1
1
2
3
3
1
1
1
2
1
2
1
1
10
1
1
1
1
1
1
1
2
1
5
1
2
2
3
2
1
1
6
1
1
1
7
2
8
3
1
2
1
1
1
1
1
1
6
1
4
1
1
1
1
1
1
1
2
1
1
10
1
1
4
1
1
3
1
3
2
1
1
1
1
4
4
3
1
5
3
1
1
4
4
1
1
1
2
1
1
7
29
2
1
1
1
5
1
1
1
2
1
1
6
9
1
1
4
2
4
5
1
4
2
13
6
3
5
2
1
1
1
1
3
1
2
1
2
1
6
1
1
1
2
1
1
1
4
8
4
1
1
1
11
4
1
1
2
1
6
1
3
4
1
2
2
1
1
6
4
3
2
1
1
2
1
10
9
2
2
5
2
2
1
1
1
1
1
1
1
1
1
1
1
1
3
3
11
1
1
1
4


In [10]:
def returnFilesAndLabelsFromSessionDict(d):
    files = []
    genders = []
    for id_num in d.keys():
        files.append(d[id_num].file)
        genders.append((d[id_num].gender, d[id_num].patient))
    return files, genders

In [11]:
testPatientFiles = []
testLabels = []
trainPatientFiles = []
trainLabels = []
for patient in sessionDict.keys():
    testSessionToAdd = np.random.choice(list(sessionDict[patient].keys()))
    for session in sessionDict[patient].keys():
        files, genders = returnFilesAndLabelsFromSessionDict(sessionDict[patient][session])
        if session == testSessionToAdd:
            testPatientFiles += files
            testLabels += genders
        else:
            trainPatientFiles += files
            trainLabels += genders

In [12]:
len(testPatientFiles), len(trainPatientFiles)

(652, 2332)

In [13]:
len(allPatients)

318

In [14]:
len(genders)

10

In [15]:
reload(er)
testEnsembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", max_num_samples=5, edf_tokens=testPatientFiles, n_process=4, labels=testLabels)[:]
trainEnsembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", max_num_samples=5, edf_tokens=trainPatientFiles, n_process=4, labels=trainLabels)[:]

Starting 4 processes
retrieving: 0
read 6, less than 9 requested!!!
read 6, less than 9 requested!!!
read 6, less than 9 requested!!!
retrieving: 10
retrieving: 20
retrieving: 30
retrieving: 40
retrieving: 50
retrieving: 60
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
retrieving: 70
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than 1005 requested!!!
read 1000, less than

read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
retrieving: 1130
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 requested!!!
read 5, less than 9 requested!!!
read 1600, less than 1605 requested!!!
read 5, less than 9 requested!!!
read 1600, less than 1605 requested!!!
read 5, less than 9 requested!!!
read 1600, less than 1605 requested!!!
read 1600, less than 1605 

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 2130
retrieving: 2140
retrieving: 2150
retrieving: 2160
retrieving: 2170
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
retrieving: 2180
retrieving: 2190
retrieving: 2200
retrieving: 2210
retrieving: 2220
retrieving: 2230
retrieving: 2240
retrieving: 2250
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
retrieving: 3230
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 3240
retrieving: 3250
read 7, less than 9 requested!!!
read 7, less than 9 requested!!!
read 7, less than 9 requested!!!
Starting 4 processes
retrieving: 0
retrieving: 10
retrieving: 20
retrieving: 30
retrieving: 40
retrieving: 50
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 6, less than 9 requested!!!
read 6, less than 9 requested!!!
retrieving: 60
read 6, less than 9 requested!!!
retrieving: 70
retrieving: 80
retrieving: 90
retrieving: 100
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
retrieving: 110
retrieving: 120
retrieving: 130
read 5, less than 9 requested!!!
read 5, less than 9 requested!!

retrieving: 880
read 7, less than 9 requested!!!
read 7, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
retrieving: 890
retrieving: 900
retrieving: 910
retrieving: 920
retrieving: 930
retrieving: 940
retrieving: 950
retrieving: 960
retrieving: 970
retrieving: 980
retrieving: 990
retrieving: 1000
retrieving: 1010
retrieving: 1020
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
retrieving: 1030
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 req

...

read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 8320
retrieving: 8330
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
retrieving: 8340
retrieving: 8350
retrieving: 8360
retrieving: 8370
retrieving: 8380
retrieving: 8390
retrieving: 8400
retrieving: 8410
retrieving: 8420
retrieving: 8430
retrieving: 8440
retrieving: 8450
retrieving: 8460
retrieving: 8470
retrieving: 8480
retrieving: 8490
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
retrieving: 8500
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, 

read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
retrieving: 9100
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
retrieving: 9890
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 9900
read 8, less than 9 requested!!!
read 8, less th

retrieving: 10350
retrieving: 10360
retrieving: 10370
retrieving: 10380
retrieving: 10390
read 7, less than 9 requested!!!
read 7, less than 9 requested!!!
read 7, less than 9 requested!!!
retrieving: 10400
retrieving: 10410
retrieving: 10420
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
retrieving: 10430
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
retrieving: 10440
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 5, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
retrieving: 10450
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 7, less than 9 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 7, less than 9 requested!!!
read 1024, less than 1030 requested!!!
read 7,

read 8, less than 9 requested!!!
read 1024, less than 1030 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
retrieving: 11170
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retr

In [16]:
len(trainEnsembler)

11633

In [17]:
trainEnsembler, validEnsembler = train_test_split(trainEnsembler, test_size=0.1)

In [45]:
reload(dg)
testDataGen = dg.DataGenMultipleLabels(testEnsembler, batch_size=256, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False)
validDataGen = dg.DataGenMultipleLabels(validEnsembler, batch_size=256, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False)
trainDataGen = dg.DataGenMultipleLabels(trainEnsembler, batch_size=64, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False, shuffle=False)

In [46]:
len(testDataGen), len(trainDataGen)

(13, 164)

In [47]:
x, y = testDataGen[0]

In [48]:
x.shape

(256, 21, 500, 1)

In [49]:
y

[array([[0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0

In [60]:
x, y = cnn_models.inception_like_pre_layers(input_shape=(21, 500, 1), num_filters=10, dropout=0.5)

In [61]:
y_gender = Dense(2, activation="softmax", name="gender")(y)
y_patient = Dense(len(allPatients), activation="softmax", name="patient")(y)

In [62]:
model = Model(inputs=x, outputs=[y_gender, y_patient])

In [63]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 20, 499, 10)  50          input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 19, 498, 10)  100         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 18, 497, 10)  170         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_73 

In [64]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model, 2)

In [65]:
adam = keras.optimizers.Adam(lr=0.00005)
model_gender_save = keras.callbacks.ModelCheckpoint("genderPredictor.h5", monitor="val_gender_loss", save_best_only=True, mode="min", verbose=True)
model_patient_save = keras.callbacks.ModelCheckpoint("patientPredictor.h5", monitor="val_patient_loss", save_best_only=True, mode="min", verbose=True)


earlyStopsOnGender = keras.callbacks.EarlyStopping(monitor="val_gender_loss", mode="min", patience=5, verbose=True)
earlyStopsOnPatient = keras.callbacks.EarlyStopping(monitor="val_patient_loss", mode="min", patience=5, verbose=True)
cb = [model_gender_save, model_patient_save, earlyStopsOnPatient]

model.compile(adam, loss=["categorical_crossentropy", "categorical_crossentropy"], metrics=["categorical_accuracy"], loss_weights=[0.5, 0.5])

In [66]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 21, 500, 1)   0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 21, 500, 1)   0           input_4[0][0]                    
__________________________________________________________________________________________________
model_7 (Model)                 [(None, 2), (None, 3 15785770    lambda_7[0][0]                   
                                                                 lambda_8[0][0]                   
__________

In [67]:
history = model.fit_generator(trainDataGen, epochs=100, validation_data=validDataGen, callbacks=cb)

Epoch 1/100
164/164 [==============================] - 52s 318ms/step - loss: 3.0509 - gender_loss: 0.8373 - patient_loss: 5.2644 - gender_categorical_accuracy: 0.5536 - patient_categorical_accuracy: 0.0792 - val_loss: 2.7075 - val_gender_loss: 0.7180 - val_patient_loss: 4.6971 - val_gender_categorical_accuracy: 0.5988 - val_patient_categorical_accuracy: 0.0765

Epoch 00001: val_gender_loss improved from inf to 0.71801, saving model to genderPredictor.h5

Epoch 00001: val_patient_loss improved from inf to 4.69707, saving model to patientPredictor.h5
Epoch 2/100
164/164 [==============================] - 31s 187ms/step - loss: 2.7296 - gender_loss: 0.8328 - patient_loss: 4.6264 - gender_categorical_accuracy: 0.5729 - patient_categorical_accuracy: 0.1188 - val_loss: 2.5808 - val_gender_loss: 0.6937 - val_patient_loss: 4.4678 - val_gender_categorical_accuracy: 0.5971 - val_patient_categorical_accuracy: 0.0902

Epoch 00002: val_gender_loss improved from 0.71801 to 0.69370, saving model to 

Epoch 17/100
164/164 [==============================] - 31s 188ms/step - loss: 1.5153 - gender_loss: 0.6911 - patient_loss: 2.3395 - gender_categorical_accuracy: 0.6593 - patient_categorical_accuracy: 0.4283 - val_loss: 2.5332 - val_gender_loss: 0.6508 - val_patient_loss: 4.4155 - val_gender_categorical_accuracy: 0.6177 - val_patient_categorical_accuracy: 0.1460

Epoch 00017: val_gender_loss did not improve from 0.62770

Epoch 00017: val_patient_loss did not improve from 4.20206
Epoch 18/100
164/164 [==============================] - 31s 189ms/step - loss: 1.4691 - gender_loss: 0.6885 - patient_loss: 2.2497 - gender_categorical_accuracy: 0.6662 - patient_categorical_accuracy: 0.4467 - val_loss: 2.4620 - val_gender_loss: 0.6669 - val_patient_loss: 4.2570 - val_gender_categorical_accuracy: 0.6151 - val_patient_categorical_accuracy: 0.1529

Epoch 00018: val_gender_loss did not improve from 0.62770

Epoch 00018: val_patient_loss did not improve from 4.20206
Epoch 19/100
164/164 [==========

In [68]:
model.fit_generator(trainDataGen, epochs=100, validation_data=validDataGen, callbacks=cb)

Epoch 1/100
164/164 [==============================] - 31s 187ms/step - loss: 0.9848 - gender_loss: 0.6251 - patient_loss: 1.3445 - gender_categorical_accuracy: 0.7003 - patient_categorical_accuracy: 0.6540 - val_loss: 2.4848 - val_gender_loss: 0.6176 - val_patient_loss: 4.3521 - val_gender_categorical_accuracy: 0.6649 - val_patient_categorical_accuracy: 0.1418

Epoch 00001: val_gender_loss improved from 0.62770 to 0.61758, saving model to genderPredictor.h5

Epoch 00001: val_patient_loss did not improve from 4.03548
Epoch 2/100
164/164 [==============================] - 30s 185ms/step - loss: 0.9554 - gender_loss: 0.6158 - patient_loss: 1.2950 - gender_categorical_accuracy: 0.7040 - patient_categorical_accuracy: 0.6688 - val_loss: 2.4794 - val_gender_loss: 0.6764 - val_patient_loss: 4.2824 - val_gender_categorical_accuracy: 0.6280 - val_patient_categorical_accuracy: 0.1632

Epoch 00002: val_gender_loss did not improve from 0.61758

Epoch 00002: val_patient_loss did not improve from 4.

In [69]:
genderPredictor = keras.models.load_model("genderPredictor.h5")
genderPredictor = multi_gpu_model(genderPredictor, 2)

In [71]:
len(testEnsembler)

3259

In [78]:
ypred = genderPredictor.predict(np.stack([data[0].reshape(500,21,1).transpose(1,0,2) for data in testEnsembler]))

In [81]:
genderPred = ypred[0]

In [82]:
accuracy_score(genderPred.argmax(1), [data[1][0] for data in testEnsembler])

0.613685179502915

In [84]:
accuracy_score(ypred[1].argmax(1), [data[1][1] for data in testEnsembler])

0.053390610616753605

In [83]:
roc_auc_score(genderPred.argmax(1), [data[1][0] for data in testEnsembler])

0.6321110176987276